# Atopy Grading - Train 

In [9]:
# !pip install tensorflow

In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
import h5py
import math
from random import randint, choice
import glob
import cv2

from scipy.ndimage import rotate
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

import keras
from keras.models import Model
from keras.layers import Dense, Activation, Input, Concatenate, Dropout, BatchNormalization, ZeroPadding2D

# print('TensorFlow version: ', tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## HyperParameters

In [2]:
##################################################
Evaluator = 'A'   ## Median  
atopy_type = '7.IGA' ##
# 1.Erythema
# 2.Edema_Papule 
# 3.Excoriation 
# 4.Lichenification
# 5.Oozing_Crusting
# 6.Dryness
# 7.IGA

normalize = 'on' ##
augment = 'on' ##
patience = 20
# label_smooth = 0.2

epoch = 200
batch_size = 32  ## 32
input_shape = [400,400,3] 

option = '_Aug_' # _aug_ # _focal_loss_0
model_names = ['InceptionResNetV2'] #  'ResNest50', ResNest101','ResNest200','ResNest269' / 'InceptionV3'
##################################################

train_path = '../Dataset/Image_re/Train/' + Evaluator + '/' + atopy_type + '/'
save_name = model_names[0] + '_' + Evaluator + '_' + atopy_type + option + '.h5'

model_path = '../Result/model/' + save_name 

print(model_path)

../Result/model/acc/InceptionResNetV2_A_new_6.Dryness_Aug_210115.h5


## Load Dataset

In [3]:
""" Train """
grade0_path = glob.glob(train_path + 'Grade0/*.jpg')
grade1_path = glob.glob(train_path + 'Grade1/*.jpg')
grade2_path = glob.glob(train_path + 'Grade2/*.jpg')
grade3_path = glob.glob(train_path + 'Grade3/*.jpg')

grade0_list, grade1_list, grade2_list, grade3_list = [], [], [], []
label0_list, label1_list, label2_list, label3_list = [], [], [], []


for g0 in grade0_path:
    img0 = cv2.imread(g0)
    ## resize
    image0 = cv2.resize(img0, dsize=(input_shape[0], input_shape[1]), interpolation = cv2.INTER_CUBIC)
    if (normalize == 'on') and (augment != 'on'):
        image0 = image0 / 255
    grade0_list.append(image0)
    label0_list.append(0)
    
for g1 in grade1_path:
    img1 = cv2.imread(g1)
    ## resize
    image1 = cv2.resize(img1, dsize=(input_shape[0], input_shape[1]), interpolation = cv2.INTER_CUBIC)
    if (normalize == 'on') and (augment != 'on'):
        image1 = image1 / 255
    grade1_list.append(image1)    
    label1_list.append(1)

for g2 in grade2_path:
    img2 = cv2.imread(g2)
    ## resize
    image2 = cv2.resize(img2, dsize=(input_shape[0], input_shape[1]), interpolation = cv2.INTER_CUBIC)
    if (normalize == 'on') and (augment != 'on'):
        image2 = image2 / 255
    grade2_list.append(image2)    
    label2_list.append(2)
        
for g3 in grade3_path:
    img3 = cv2.imread(g3)
    ## resize
    image3 = cv2.resize(img3, dsize=(input_shape[0], input_shape[1]), interpolation = cv2.INTER_CUBIC)
    if (normalize == 'on') and (augment != 'on'):
        image3 = image3 / 255
    grade3_list.append(image3)        
    label3_list.append(3)

print(np.shape(grade0_list))
print(np.shape(grade1_list))
print(np.shape(grade2_list))
print(np.shape(grade3_list))

print("** Process Done ***")

(4402, 400, 400, 3)
(997, 400, 400, 3)
(473, 400, 400, 3)
(262, 400, 400, 3)
** Process Done ***


## Augmentation

In [4]:
grade_list = [len(grade0_list), len(grade1_list), len(grade2_list), len(grade3_list)]

g0_aug_num = int(round(grade_list[np.argmax(grade_list)]/len(grade0_list),0))
g1_aug_num = int(round(grade_list[np.argmax(grade_list)]/len(grade1_list),0))
g2_aug_num = int(round(grade_list[np.argmax(grade_list)]/len(grade2_list),0))
g3_aug_num = int(round(grade_list[np.argmax(grade_list)]/len(grade3_list),0))

print(len(grade0_list), ' x ', g0_aug_num)
print(len(grade1_list), ' x ', g1_aug_num)
print(len(grade2_list), ' x ', g2_aug_num)
print(len(grade3_list), ' x ', g3_aug_num)

4402  x  1
997  x  4
473  x  9
262  x  17


In [5]:
from imgaug import augmenters as iaa

""" Augmentation Methods"""
CT_aug = iaa.ChangeColorTemperature((4000, 7500))
seq = iaa.Sequential([
                    iaa.Fliplr(0.5),
                    iaa.Flipud(0.75),
                    iaa.GammaContrast((0.8, 1.1)),
                    iaa.Sharpen((0.0, 0.3)), 
#                     iaa.LinearContrast((0.75, 1.25)),
#                     iaa.CoarseDropout((0.0, 0.05), size_percent=(0.2, 0.25)),
#                     iaa.GaussianBlur((0, 1.0)),
#                     iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
                    iaa.PiecewiseAffine(scale=(0.01, 0.05)), # some images distort local areas with varying strength.
                    iaa.PerspectiveTransform(scale=(0.01, 0.15))
#                     iaa.ElasticTransformation(alpha=20, sigma=5)
                    ], 
                    random_order=False)

""" Apply """
aug0_list, aug1_list, aug2_list, aug3_list = [], [], [], []
label_aug0_list, label_aug1_list, label_aug2_list, label_aug3_list = [], [], [], []

## Grade 0
if g0_aug_num > 1:
    print("Grade0 Augmentation")
    for g0_num in range(g0_aug_num):        
        for grade0 in grade0_list: 
            aug_CT0 = CT_aug(image=grade0)
            aug = seq(image=aug_CT0)
            if normalize == 'on':
                aug0_list.append(aug / 255)
            else:
                aug0_list.append(aug)
            label_aug0_list.append(0)
else:
    for grade0 in grade0_list: 
        aug_CT0 = CT_aug(image=grade0)
        aug = seq(image=aug_CT0)
        if normalize == 'on':
            aug0_list.append(aug / 255)
        else:
            aug0_list.append(aug)
    label_aug0_list = label0_list

## Grade 1    
if g1_aug_num > 1:
    print("Grade1 Augmentation")
    for g1_num in range(g1_aug_num):        
        for grade1 in grade1_list:
            aug_CT0 = CT_aug(image=grade1)
            aug = seq(image=aug_CT0)
            if normalize == 'on':
                aug1_list.append(aug / 255)
            else:
                aug1_list.append(aug)
            label_aug1_list.append(1)
else:
    for grade1 in grade1_list: 
        aug_CT0 = CT_aug(image=grade1)
        aug = seq(image=aug_CT0)
        if normalize == 'on':
            aug1_list.append(aug / 255)
        else:
            aug1_list.append(aug)
    label_aug1_list = label1_list

## Grade 2    
if g2_aug_num > 1:
    print("Grade2 Augmentation")
    for g2_num in range(g2_aug_num):        
        for grade2 in grade2_list:
            aug_CT0 = CT_aug(image=grade2)
            aug = seq(image=aug_CT0)
            if normalize == 'on':
                aug2_list.append(aug / 255)
            else:
                aug2_list.append(aug)
            label_aug2_list.append(2)
else:
    for grade2 in grade2_list: 
        aug_CT0 = CT_aug(image=grade2)
        aug = seq(image=aug_CT0)
        if normalize == 'on':
            aug2_list.append(aug / 255)
        else:
            aug2_list.append(aug)
    label_aug2_list = label2_list
    
## Grade 3
if g3_aug_num > 1:
    print("Grade3 Augmentation")
    for g3_num in range(g3_aug_num):        
        for grade3 in grade3_list:
            aug_CT0 = CT_aug(image=grade3)
            aug = seq(image=aug_CT0)
            if normalize == 'on':
                aug3_list.append(aug / 255)
            else:
                aug3_list.append(aug)
            label_aug3_list.append(3)
else:
    for grade3 in grade3_list: 
        aug_CT0 = CT_aug(image=grade3)
        aug = seq(image=aug_CT0)
        if normalize == 'on':
            aug3_list.append(aug / 255)
        else:
            aug3_list.append(aug)
    label_aug3_list = label3_list

del grade0_list, grade1_list, grade2_list, grade3_list
del label0_list, label1_list, label2_list, label3_list

print(np.shape(aug0_list))
print(np.shape(aug1_list))
print(np.shape(aug2_list))
print(np.shape(aug3_list))

Grade1 Augmentation
Grade2 Augmentation
Grade3 Augmentation
(4402, 400, 400, 3)
(3988, 400, 400, 3)
(4257, 400, 400, 3)
(4454, 400, 400, 3)


## Concatenation for Dataset Preparation

In [6]:
""" Train """
if augment == 'on':
    X_train = np.concatenate((aug0_list, aug1_list, aug2_list, aug3_list), axis = 0)
    y_train_ = np.concatenate((label_aug0_list, label_aug1_list, label_aug2_list, label_aug3_list), axis = 0)

    del aug0_list, aug1_list, aug2_list, aug3_list
    del label_aug0_list, label_aug1_list, label_aug2_list, label_aug3_list
    
else:
    X_train = np.concatenate((grade0_list, grade1_list, grade2_list, grade3_list), axis = 0)
    y_train_ = np.concatenate((label0_list, label1_list, label2_list, label3_list), axis = 0)

    del grade0_list, grade1_list, grade2_list, grade3_list
    del label0_list, label1_list, label2_list, label3_list


np.random.seed(0)  
np.random.shuffle(X_train)
np.random.seed(0)  
np.random.shuffle(y_train_)

""" One-hot encodding """
y_train = keras.utils.to_categorical(y_train_,4) #


""" Class Weight """
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train_),y_train_)
class_weight_keras = {0: class_weights[0], 1: class_weights[1], 2: class_weights[2], 3: class_weights[3]}
print(class_weight_keras)

print(np.shape(X_train))
print(np.shape(y_train))

{0: 0.9712062698773285, 1: 1.0720285857572718, 2: 1.004287056612638, 3: 0.9598675348001796}
(17101, 400, 400, 3)
(17101, 4)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2 3], y=[2 2 3 ... 2 2 0] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


## Define Model

In [7]:
import keras.applications.inception_resnet_v2 as InceptionResNetV2

input_tensor = Input(shape=(input_shape[0], input_shape[1], input_shape[2]))


""" InceptionResNetV2 """
model = InceptionResNetV2.InceptionResNetV2(include_top= True, input_tensor=input_tensor, input_shape=None,
                                                                      weights = None, pooling= None, classes=4

# model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




' InceptionV3 (Pretrained Model) '

## Callbacks

In [8]:
from keras.callbacks import  ModelCheckpoint, CSVLogger, ReduceLROnPlateau

############### ############### ############### ############### ###############
model.compile(loss='categorical_crossentropy',              
              optimizer = keras.optimizers.Adam(lr=0.001, decay=1e-6),
              metrics = ['acc']
              )


checkpoint = keras.callbacks.ModelCheckpoint(
                  filepath = model_path,
                  monitor='val_loss',  
                   mode = 'min',
                   save_best_only=True, 
                   verbose=1)
############### ############### ############### ############### ###############
reducer = ReduceLROnPlateau(monitor='loss', factor=.8, patience=5, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=patience, verbose=1) # 'val_auc'

callback = [checkpoint, reducer] ##

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


## Train Model

In [9]:
import keras.backend.tensorflow_backend as K

""" Training Model """
with K.tf.device('/gpu:0'):
    trained_model = model.fit(X_train, y_train,
                               validation_split = 0.2,
                              batch_size = batch_size, 
                              epochs = epoch, 
                              callbacks = callback,   
                              class_weight = class_weight_keras,
                              shuffle = True,
                              verbose=2)

print("\n***** Training Model Complete *****")


Train on 13680 samples, validate on 3421 samples
Epoch 1/200
 - 467s - loss: 1.0951 - acc: 0.5216 - val_loss: 1.9908 - val_acc: 0.2979

Epoch 00001: val_acc improved from -inf to 0.29787, saving model to ../Result/model/acc/InceptionResNetV2_A_new_6.Dryness_Aug_210115.h5

Epoch 00001: val_loss improved from inf to 1.99077, saving model to ../Result/model/loss/InceptionResNetV2_A_new_6.Dryness_Aug_210115.h5
Epoch 2/200
 - 410s - loss: 0.9413 - acc: 0.5878 - val_loss: 1.0517 - val_acc: 0.5978

Epoch 00002: val_acc improved from 0.29787 to 0.59778, saving model to ../Result/model/acc/InceptionResNetV2_A_new_6.Dryness_Aug_210115.h5

Epoch 00002: val_loss improved from 1.99077 to 1.05175, saving model to ../Result/model/loss/InceptionResNetV2_A_new_6.Dryness_Aug_210115.h5
Epoch 3/200
 - 410s - loss: 0.8777 - acc: 0.6121 - val_loss: 0.8446 - val_acc: 0.6177

Epoch 00003: val_acc improved from 0.59778 to 0.61766, saving model to ../Result/model/acc/InceptionResNetV2_A_new_6.Dryness_Aug_21011

KeyboardInterrupt: 

## Plot Training Process

In [10]:
""" Accuracy Graph """
fig, acc_ax = plt.subplots()
loss_ax = acc_ax 

acc_ax.plot(trained_model.history['acc'], 'b', label='train acc')
acc_ax.plot(trained_model.history['val_acc'], 'g', label='val acc')
loss_ax.plot(trained_model.history['loss'], 'y', label='train loss')
loss_ax.plot(trained_model.history['val_loss'], 'r', label='val loss')


acc_ax.set_xlabel('Epoch')
acc_ax.set_ylabel('Accuracy')

acc_ax.legend(loc='upper right')

plt.show()